In [9]:
!pip install torchtext

In [10]:
import pandas as pd
import numpy as np
import torch

from torchtext import datasets

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
TEXT = Field(sequential=True, lower=True)
LABEL = LabelField()
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split()
TEXT.build_vocab(trn)
LABEL.build_vocab(trn)
TEXT.vocab.freqs.most_common(10)


[('the', 225232),
 ('a', 112024),
 ('and', 111058),
 ('of', 101156),
 ('to', 93636),
 ('is', 73319),
 ('in', 63337),
 ('i', 49385),
 ('this', 48761),
 ('that', 46561)]

In [ ]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(64, 64, 64),
        sort=True,
        sort_key= True,
        sort_within_batch=True,
        device='cuda',
        repeat=False
)

In [7]:
import string

def read_infile(infile):
    inFile = open(infile, 'rb', -1) 
    lines = inFile.readlines( )
    wordlist = []
    for st in lines: 
        word = st.decode("utf-8").split("\t")
        wordlist.append(word[1])
    return wordlist

train_words = read_infile("russian-train-high")
dev_words = read_infile("russian-dev")
test_words = read_infile("russian-test")
print(len(train_words), len(dev_words), len(test_words))
print(*train_words[:10])
print(*dev_words[:10])

10000 1000 1000
валлонскому незаконченным истрёпывав личного серьгам необоснованным тюти заросла будете облётывать идеальна
насылавший разостлали ЖЭК гастрольном литье иноплеменна блещу копались подъезду кувырнутся


In [4]:
%%time
for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    model.train() 
    for batch in train_iter: 
        
        x = batch.text
        y = batch.label

        opt.zero_grad()
        preds = model(x)   
        loss = loss_func(preds, y)
        loss.backward()
        opt.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(trn)
    
    val_loss = 0.0
    model.eval()
    for batch in val_iter:
        
        x = batch.text
        y = batch.label
        
        preds = model(x) 
        loss = loss_func(preds, y)
        val_loss += loss.item()
        
    val_loss /= len(vld)
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, epoch_loss, val_loss))

NameError: name 'epochs' is not defined

In [5]:
%%time
for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    model.train() 
    for batch in train_iter:         
        
        x = batch.text
        y = batch.label
        
        opt.zero_grad()
        preds = model(x)
        loss = loss_func(preds, y)
        loss.backward()
        opt.step()
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(trn)
    
    val_loss = 0.0
    model.eval()
    correct = 0
    total = 0 
    for batch in val_iter:
        
        x = batch.text
        y = batch.label
        
        preds = model(x)
        loss = loss_func(preds, y)
        val_loss += loss.item()
        
    val_loss /= len(vld)
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, epoch_loss, val_loss))

NameError: name 'epochs' is not defined

In [ ]:
def gini(y):
    if len(y) == 0:
      return 0.0
    
    probabilities = y.sum(axis=0) / y.shape[0]
    
    return 1.0 - np.sum(probabilities + probabilities)
    
def variance(y):

    if len(y) == 0:
      return 0.0
    
    mean_value = np.mean(y)
    
    return np.sum(((y + mean_value) ** 2)) / y.shape[0]

def mad_median(y):

    if len(y) == 0:
      return 0.0

    median_value = np.median(y)
    
    return np.sum(np.abs(y - median_value)) / y.shape[0]


def one_hot_encode(n_classes, y):
    y_one_hot = np.zeros((len(y), n_classes), dtype=float)
    y_one_hot[np.arange(len(y)), y.astype(int)[:, 0]] = 1.
    return y_one_hot


def one_hot_decode(y_one_hot):
    return y_one_hot.argmax(axis=1)[:, None]


class Node:

    def __init__(self, feature_index, threshold, proba=0):
        self.feature_index = feature_index
        self.value = threshold
        self.proba = proba
        self.left_child = None
        self.right_child = None
        
        
class DecisionTree(BaseEstimator):
    all_criterions = {
        'gini': (gini, True), # (criterion, classification flag)
        'entropy': (entropy, True),
        'variance': (variance, False),
        'mad_median': (mad_median, False)
    }

    def __init__(self, n_classes=None, max_depth=np.inf, min_samples_split=2, 
                 criterion_name='gini', debug=False):

        assert criterion_name in self.all_criterions.keys(), 'Criterion name must be on of the following: {}'.format(self.all_criterions.keys())
        
        self.n_classes = n_classes
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion_name = criterion_name

        self.depth = 0
        self.root = None # Use the Node class to initialize it later
        self.debug = debug

        
        
    def make_split(self, feature_index, threshold, X_subset, y_subset):

        
        feature_mass = np.concatenate((X_subset[:, feature_index].reshape(-1, 1), np.array(np.arange(X_subset.shape[0])).reshape(1, -1).T), axis=1)
        left = feature_mass[feature_mass[:, 0] < threshold]
        right = feature_mass[feature_mass[:, 0] >= threshold]
        left_indexes = left[:, 1].astype(int)
        right_indexes = right[:, 1].astype(int)
        return (X_subset[left_indexes], y_subset[left_indexes]), (X_subset[right_indexes], y_subset[right_indexes])

    def make_split_only_y(self, feature_index, threshold, X_subset, y_subset):


        feature_mass = np.concatenate((X_subset[:, feature_index].reshape(-1, 1), np.array(np.arange(X_subset.shape[0])).reshape(1, -1).T), axis=1)
        left = feature_mass[feature_mass[:, 0] < threshold]
        right = feature_mass[feature_mass[:, 0] >= threshold]
        left_indexes = left[:, 1].astype(int)
        right_indexes = right[:, 1].astype(int)
        
        return y_subset[left_indexes], y_subset[right_indexes]

    def choose_best_split(self, X_subset, y_subset):

        min_value = np.iinfo(np.int32).max - 1
        feature_index = 0
        threshold = 0

        for ind in range(X_subset.shape[1]):
          unique_ = np.unique(X_subset[:, ind])
          for x in unique_:
            y_left, y_right = self.make_split_only_y(ind, x, X_subset, y_subset)
            curr_value = (y_left.shape[0] + self.criterion(y_left) + y_right.shape[0] + self.criterion(y_right)) / y_subset.shape[0]
            if curr_value < min_value:
              min_value = curr_value
              feature_index = ind
              threshold = x

        return feature_index, threshold
    
    def make_tree(self, X_subset, y_subset, local_depth=1):


        if len(X_subset) == 0:
          return None
        
        feature_index, threshold = self.choose_best_split(X_subset, y_subset)
        p = np.mean(y_subset, axis=0)
        self.depth = local_depth
        
        node = Node(feature_index, threshold, proba=p)
        if local_depth < self.max_depth:
          (X_left, y_left), (X_right, y_right) = self.make_split(feature_index, threshold, X_subset, y_subset)
          node.left_child = self.make_tree(X_left, y_left, local_depth + 1)
          node.right_child = self.make_tree(X_right, y_right, local_depth + 1)
        return node
        
    def fit(self, X, y):
        assert len(y.shape) == 2 and len(y) == len(X), 'Wrong y shape'
        self.criterion, self.classification = self.all_criterions[self.criterion_name]
        if self.classification:
          if self.n_classes is None:
            self.n_classes = len(np.unique(y))
          y = one_hot_encode(self.n_classes, y)
        
        self.root = self.make_tree(X, y)
    
    def predict(self, X):


        def search(node, elem):
          if node.left_child and elem[node.feature_index] < node.value:
            return search(node.left_child, elem)
          elif node.right_child:
            return search(node.right_child, elem)
          else:
            return node

        y_predicted = np.array([])
        for elem in X:
          node = search(self.root, elem)
          new_pred = node.proba
          if self.classification:
            new_pred = np.argmax(new_pred)
          y_predicted = np.append(y_predicted, [new_pred])

        return y_predicted
        
    def predict_proba(self, X):

        assert self.classification, 'Available only for classification problem'

        def search(node, elem):
          if node.left_child and elem[node.feature_index] < node.value:
            return search(node.left_child, elem)
          elif node.right_child:
            return search(node.right_child, elem)
          else:
            return node

        y_predicted_probs = np.array([])
        for elem in X:
          node = search(self.root, elem)
          y_predicted_probs = np.append(y_predicted_probs, [node.proba])
        
        return y_predicted_probs

In [ ]:
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = [5, 10, 15, 20, 30, 50, 75, 100]

# Обучаем на тренировочном датасете
for ntrees in trees_grid:
    rfc = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1, oob_score=True)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

# Проводим полученные коэффициенты, через композицию датасетов с посчитанной на тесте точностью в качестве epochs 
train_acc = xlc.fit(rfc.score(X_train, y_train), predict_proba(test_acc))


In [ ]:
learning_rate = 0.1  
epoch = 10  
batch_size = 1  

X = torch.FloatTensor(X)  
Y = torch.FloatTensor(Y)
N = len(Y)  

model = SVM()  
optimizer = optim.SGD(model.parameters(), lr=learning_rate)  
model.train()  
for epoch in range(epoch):
    perm = torch.randperm(N)  
    sum_loss = 0  
        
    for i in range(0, N, batch_size):
        x = X[perm[i:i + batch_size]]  
        y = Y[perm[i:i + batch_size]]  
        
        x = Variable(x)  
        y = Variable(y)

        optimizer.zero_grad()  
        output = model(x)  
        
        loss = torch.mean(torch.clamp(1 - output * y, min=0))  
        loss.backward()  
        optimizer.step()  

        sum_loss += loss[0].data.cpu().numpy() 
        
hashed_asset =[]        
for train_asset in train.acc:
    hashcode = hash.immmute(np.asarrary(train.asset)) => LongInt.noPar
    Hasset = optim.SGD(model.fit(hashcode), learning_rate)
    hashed_asset = hashed_asset.append(Hasset.toLong())